In [16]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [17]:
url = 'https://thetvdb.com/series/house-hunters-international/allseasons/official'
page = requests.get(url)
page

<Response [200]>

In [18]:
#Scrape one result each to start
page = requests.get(url)
soup = BeautifulSoup(page.text,'lxml')

ep_label = soup.find('h4',class_ ="list-group-item-heading").text.strip().split('\n\n')[1].strip()
ep_summary = soup.find('p').text.strip()
air_date = soup.find('ul',class_ = 'list-inline text-muted').find('li').text.strip()

In [19]:
#Get lists
ep_label = soup.find_all('h4',class_ ="list-group-item-heading")
ep_summary = soup.find_all('p')
air_date = soup.find_all('ul',class_ = 'list-inline text-muted')

print('# of episode labels scraped: ', len(ep_label))
print('# of episode summaries scraped: ', len(ep_summary))
print('# of air dates scraped: ', len(air_date))

# of episode labels scraped:  2432
# of episode summaries scraped:  2432
# of air dates scraped:  2432


In [20]:
print(ep_label[1000].text)
print(air_date[1000].text)
print(ep_summary[1000].text)


S77E12

                                    Seeking Italian Charm in Iesi
                                


April 30, 2015
HGTV

A family's move to Iesi, Italy, is filled with both excitement and uncertainty as they look for a place with oodles of Italian charm and enough room to homeschool four children.



In [21]:
df = pd.DataFrame(
    list(zip(ep_label, ep_summary, air_date)), columns=['ep_label', 'ep_summary', 'air_date']
)
df.shape

(2432, 3)

In [22]:
df.head()

,ep_label,ep_summary,air_date
0,"[\n, [S01E01], \n, [\n ...","[After attending college in the United States,...","[\n, [February 20, 2006], \n, [HGTV], \n]"
1,"[\n, [S01E02], \n, [\n ...",[Real estate agent Michelle Owens adores her l...,"[\n, [April 17, 2006], \n, [HGTV], \n]"
2,"[\n, [S01E03], \n, [\n ...","[Gary Howorka and his girlfriend, Rebecca Ande...","[\n, [September 4, 2006], \n, [HGTV], \n]"
3,"[\n, [S01E04], \n, [\n ...","[After living with his sister for five years, ...","[\n, [June 2, 2006], \n, [HGTV], \n]"
4,"[\n, [S01E05], \n, [\n ...","[Martin Michajluk and his fiancee, Eva Dakovov...","[\n, [June 23, 2006], \n, [HGTV], \n]"


In [23]:
df['ep_label'][0]

<h4 class="list-group-item-heading">
<span class="text-muted episode-label">S01E01</span>
<a href="/series/house-hunters-international/episodes/2653681">
                                    Planting New Costa Rican Roots
                                </a>
</h4>

In [24]:
df['ep_nums'] = df['ep_label'].astype(str).str.extract(r">(.*?)</span>")
df['ep_title'] = df['ep_label'].astype(str).str.split('>\n').str[3].str.strip().str.split('\n').str[0]
df['episode'] = pd.to_numeric(df['ep_nums'].str.split('E').str[1], errors = 'coerce')
df['season'] = pd.to_numeric(df['ep_nums'].str.split('E').str[0].str[1:], errors = 'coerce')
df['ep_summary'] = df['ep_summary'].astype(str).str.extract(r"<p>(.*?)</p>")
df['air_date'] = df['air_date'].astype(str).str.extract(r"<li>(.*?)</li>")
df['year'] = df['air_date'].str.split(', ').str[1]

In [25]:
df.head()

,ep_label,ep_summary,air_date,ep_nums,ep_title,episode,season,year
0,"[\n, [S01E01], \n, [\n ...","After attending college in the United States, ...","February 20, 2006",S01E01,Planting New Costa Rican Roots,1.0,1.0,2006
1,"[\n, [S01E02], \n, [\n ...",Real estate agent Michelle Owens adores her li...,"April 17, 2006",S01E02,Belize Bound,2.0,1.0,2006
2,"[\n, [S01E03], \n, [\n ...","Gary Howorka and his girlfriend, Rebecca Ander...","September 4, 2006",S01E03,Honduras Home Hunt,3.0,1.0,2006
3,"[\n, [S01E04], \n, [\n ...","After living with his sister for five years, S...","June 2, 2006",S01E04,Sao Paulo Search,4.0,1.0,2006
4,"[\n, [S01E05], \n, [\n ...","Martin Michajluk and his fiancee, Eva Dakovova...","June 23, 2006",S01E05,Picking a Place in Prague,5.0,1.0,2006


In [26]:
import spacy

nlp = spacy.load('en_core_web_sm')
df_test = pd.DataFrame({'Text':['The University of Cambridge is a collegiate research university in Cambridge, United Kingdom.', "Cambridge is ranked among the most prestigious universities in the world and currently sits as the world's second best university, and the best in Europe, according to the QS World University Rankings."]})
df_test.head()

,Text
0,The University of Cambridge is a collegiate re...
1,Cambridge is ranked among the most prestigious...


In [28]:
df_test['Entities'] = df_test['Text'].apply(lambda sent: [(ent.text, ent.start_char, ent.end_char, ent.label_) for ent in nlp(sent).ents])  
df_test['Entities'][1]

[('Cambridge', 0, 9, 'ORG'),
 ('second', 107, 113, 'ORDINAL'),
 ('Europe', 147, 153, 'LOC'),
 ('the QS World University Rankings', 168, 200, 'ORG')]

In [30]:
df['Entities'] = df['ep_summary'].astype(str).apply(lambda sent: [(ent.text, ent.start_char, ent.end_char, ent.label_) for ent in nlp(sent).ents])  
df['Entities'][1]

[('Michelle Owens', 18, 32, 'PERSON'),
 ('Charleston', 52, 62, 'GPE'),
 ('S.C.', 64, 68, 'GPE'),
 ('Belize', 95, 101, 'GPE'),
 ('six years ago', 102, 115, 'DATE'),
 ('Central American', 175, 191, 'NORP'),
 ('the Caribbean Sea', 356, 373, 'LOC'),
 ('Howard Oldham', 663, 676, 'PERSON')]

In [31]:
df.head()

,ep_label,ep_summary,air_date,ep_nums,ep_title,episode,season,year,Entities
0,"[\n, [S01E01], \n, [\n ...","After attending college in the United States, ...","February 20, 2006",S01E01,Planting New Costa Rican Roots,1.0,1.0,2006,"[(the United States, 27, 44, GPE), (27-year-ol..."
1,"[\n, [S01E02], \n, [\n ...",Real estate agent Michelle Owens adores her li...,"April 17, 2006",S01E02,Belize Bound,2.0,1.0,2006,"[(Michelle Owens, 18, 32, PERSON), (Charleston..."
2,"[\n, [S01E03], \n, [\n ...","Gary Howorka and his girlfriend, Rebecca Ander...","September 4, 2006",S01E03,Honduras Home Hunt,3.0,1.0,2006,"[(Gary Howorka, 0, 12, PERSON), (Rebecca Ander..."
3,"[\n, [S01E04], \n, [\n ...","After living with his sister for five years, S...","June 2, 2006",S01E04,Sao Paulo Search,4.0,1.0,2006,"[(five years, 33, 43, DATE), (Sao Paulo, 45, 5..."
4,"[\n, [S01E05], \n, [\n ...","Martin Michajluk and his fiancee, Eva Dakovova...","June 23, 2006",S01E05,Picking a Place in Prague,5.0,1.0,2006,"[(Martin Michajluk, 0, 16, PERSON), (Eva Dakov..."


In [34]:
#df.to_csv('hhi.csv')  